# This NB demonstrates how we can use methods from visualizations

We can use the method to plot all patterns, double click on specific patterns to highlight them in the heatmaps, and select top patterns to plot them on their own.

In [1]:
# python imports
import pandas as pd
import altair as alt

# imports to load relationships
import requests
from pandas.io.json import json_normalize

# our imports
from main import *
import visualizations as viz

## `plot_ngrams_heatmap`

First, we do the normal steps to get a ngrams dataframe.

In [2]:
root = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/"
prefix = "CRIM_Model_00"
files = ["09"] 
postfix = ".mei"
corpus = CorpusBase([root + prefix + files[0] + postfix])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=0)
mel_ngrams = model.getNgrams(df=mel, n=5)
mel_ngrams.head()

Successfully imported.


,Superius,Contratenor,Tenor,Bassus
4.0,NaN,NaN,NaN,"1, 1, 3, -2, -2"
6.0,NaN,NaN,NaN,"1, 3, -2, -2, -2"
8.0,NaN,NaN,"1, 1, 3, 1, 2","3, -2, -2, -2, -2"
10.0,NaN,NaN,"1, 3, 1, 2, -4",NaN
11.0,NaN,NaN,NaN,"-2, -2, -2, -2, -2"


Can read the documentation with `help(viz.plot_ngrams_heatmap)`

In [3]:
help(viz.plot_ngrams_heatmap)

Help on function plot_ngrams_heatmap in module visualizations:

plot_ngrams_heatmap(ngrams_df, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot a heatmap for crim-intervals getNgram's output.
    :param ngrams_df: crim-intervals getNgram's output
    :param ngrams_duration: if not None, rely on durations in the
    df to calculate the durations of the ngrams.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: the height of the final heatmap (optional)
    :return: a bar chart that displays the different patterns and their counts,
    and a heatmap with the start offsets of chosen voices / patterns



In [4]:
viz.plot_ngrams_heatmap(mel_ngrams,selected_patterns=[], voices=[])

alt.VConcatChart(...)

### Selecting patterns

We could only include some patterns in the heatmaps

First, I collected the top patterns and turned these patterns into a list.

In [5]:
# count and get the 10 most popular patterns
mel_ngrams_top_patterns_df = mel_ngrams.stack().value_counts().to_frame().head(10)
# retrieve a list to generate heatmaps
mel_ngrams_top_patterns_list = mel_ngrams_top_patterns_df.index.to_list()
mel_ngrams_top_patterns_df

,0
"2, 2, -2, -2, -2",5
"-3, 2, 2, 2, 2",5
"2, -2, -2, -2, 2",5
"1, 1, 3, 1, 2",5
"-2, -2, 2, 2, 2",5
"1, 1, -2, -2, -2",5
"1, -2, -2, -2, 2",5
"-3, 2, -2, -2, -2",4
"1, 3, 1, 2, -4",4
"2, 2, 2, -2, -2",4


In [6]:
mel_ngrams_top_patterns_list 

['2, 2, -2, -2, -2',
 '-3, 2, 2, 2, 2',
 '2, -2, -2, -2, 2',
 '1, 1, 3, 1, 2',
 '-2, -2, 2, 2, 2',
 '1, 1, -2, -2, -2',
 '1, -2, -2, -2, 2',
 '-3, 2, -2, -2, -2',
 '1, 3, 1, 2, -4',
 '2, 2, 2, -2, -2']

We would pass the list of patterns into the `patterns` parametter.

In [7]:
viz.plot_ngrams_heatmap(mel_ngrams, selected_patterns=mel_ngrams_top_patterns_list)

alt.VConcatChart(...)

**Cross-checking output from the heatmap with the ngrams dataframe**

We can see that '-2, -2, -2, -2, -2' is a really popular pattern. Let's view its start points in ngram and in the heatmap's dataframe to make sure that it has been calculated correctly

In [8]:
mel_ngrams[mel_ngrams ==  '-2, -2, -2, -2, -2' ].stack()

11.0  Bassus    -2, -2, -2, -2, -2
12.0  Bassus    -2, -2, -2, -2, -2
dtype: object

## `generate_ngrams_and_dur` allow us to create heatmap with durations!

How could we create a heatmap for ngrams in which the ngrams' durations are desplayed?

In [9]:
ngrams, ngrams_duration = viz.generate_ngrams_and_duration(model, mel, n=5)

In [10]:
help(viz.generate_ngrams_and_duration)

Help on function generate_ngrams_and_duration in module visualizations:

generate_ngrams_and_duration(model, df, n=3, exclude=['Rest'], interval_settings=('d', True, True), offsets='first')
    This method accept a model and a dataframe with the melody or notes
    and rests and generate an ngram (in columnwise and unit=0 setting)
    and a corresponding duration ngram
    :param model: an Imported Piece object.
    :param df: dataframe containing consecutive notes.
    :param n: accept any positive integers and would output ngrams of the corresponding sizes
    can't handle the n=-1 option (refer to getNgrams documentation for more)
    :param exclude: (refer to getNgrams documentation)
    :param interval_settings: (refer to getNgrams documentation)
    :param offsets: (refer to getNgrams documentation)
    :return: ngram and corresponding duration dataframe!



In [11]:
viz.plot_ngrams_heatmap(ngrams, ngrams_duration, selected_patterns=mel_ngrams_top_patterns_list , voices=[])

alt.VConcatChart(...)

## `plot_close_match_heatmap`

In [12]:
help(viz.plot_close_match_heatmap)

Help on function plot_close_match_heatmap in module visualizations:

plot_close_match_heatmap(ngrams_df, key_pattern, ngrams_duration=None, selected_patterns=[], voices=[], heatmap_width=800, heatmap_height=300)
    Plot how closely the other vectors match a selected vector.
    Uses the Levenshtein distance.
    :param ngrams_df: crim-intervals getNgram's output
    :param key_pattern: a pattern the users selected to compare other patterns with (tuple of floats)
    :param selected_pattern: the specific other vectors the users selected
    :param ngrams_duration: if None, simply output the offsets. If the users input a
    list of durations, caculate the end by adding durations with offsets and
    display the end on the heatmap accordingly.
    :param selected_patterns: list of specific patterns the users want (optional)
    :param voices: list of specific voices the users want (optional)
    :param heatmap_width: the width of the final heatmap (optional)
    :param heatmap_height: t

In [13]:
viz.plot_close_match_heatmap(mel_ngrams, '-2, -2, -2, -2, -2', selected_patterns=[], voices=[])

alt.Chart(...)

In [14]:
# use the durations calculated from above
viz.plot_close_match_heatmap(ngrams,'-2, -2, -2, -2, -2', ngrams_duration, selected_patterns=[], voices=[])

alt.Chart(...)

## `plot_comparison_heatmap`

This method can be used to plot a relationship/observation json

### For relationships

In [15]:
# data_relationships = requests.get('http://crimproject.org/data/relationships/').json()
# df_relationships = pd.json_normalize(data_relationships)
df_relationships = pd.read_csv('../CRIM_Intervals_Notebooks/relationships.csv')
files = df_relationships['model_observation.piece.piece_id'].unique()
df_relationships_test = df_relationships[df_relationships['model_observation.piece.piece_id'] == files[1]].copy()

In [16]:
#df_relationships.to_csv('../CRIM_Intervals_Notebooks/relationships.csv')
df_relationships_test.head()

,Unnamed: 0,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
4,4,http://crimproject.org/data/relationships/5/,5,Quotation,Soggetto,True,True,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,"1-38/6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6...",http://crimproject.org/data/observations/10/,10,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0017_4,Missa Baisez moy: Sanctus,"1-41/5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5..."
5,5,http://crimproject.org/data/relationships/6/,6,Mechanical transformation,Fuga,False,False,False,True,True,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,"1-38/3+6,3+6,3+6,3+6,3+6,3+6,3+6,3+6,3+6,3+6,3...",http://crimproject.org/data/observations/12/,12,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0017_4,Missa Baisez moy: Sanctus,"1-41/5,5,5,5,4-5,4-5,4-5,4-5,4-5,4-5,4-5,4-5,4..."
6,6,http://crimproject.org/data/relationships/7/,7,Non-mechanical transformation,Fuga,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,"1-4/2,1-2,1-2,1/@3,@3+@1-3,@1-3+@1,@1",http://crimproject.org/data/observations/14/,14,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0017_4,Missa Baisez moy: Sanctus,"1-10/1-2,1-2,1-2,1-2,1-2,1-2,1-2,1,1,1/@all+@1..."
7,7,http://crimproject.org/data/relationships/8/,8,Mechanical transformation,Fuga,False,False,False,True,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,"1-5/1-2,1-2,1-2,1-2,1/@all+@3,@1-3+@1-3,@1-3+@...",http://crimproject.org/data/observations/16/,16,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0017_5,Missa Baisez moy: Agnus Dei,"42-47/2,2,2,1-2,1,1/@1,@1-3,@1-4.5,@1+@1,@1-3,@1"
548,548,http://crimproject.org/data/relationships/557/,557,Quotation,NaN,True,True,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0018,Baisez moy,3/1/@2,http://crimproject.org/data/observations/1114/,1114,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,5/2/@2.5


In [17]:
help(viz.plot_comparison_heatmap)

Help on function plot_comparison_heatmap in module visualizations:

plot_comparison_heatmap(df, ema_col, main_category='musical_type', other_category='observer.name', option=1, heat_map_width=800, heat_map_height=300)
    This method plots a chart for relationships/observations dataframe retrieved from their
    corresponding json files. This chart has two bar charts displaying the count of variables
    the users selected, and a heatmap displaying the locations of the relationship.
    :param df: relationships or observations dataframe
    :param ema_col: name of the ema column
    :param main_category: name of the main category for the first bar chart.
    The chart would be colored accordingly (default='musical_type').
    :param other_category: name of the other category for the zeroth bar chart.
    (default='observer.name')
    :param heat_map_width: the width of the final heatmap (default=800)
    :param heat_map_height: the height of the final heatmap (default =300)
    :return

In [29]:
viz.plot_comparison_heatmap(df_relationships_test, 'model_observation.ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

### For observations

In [19]:
# data_observations = requests.get('http://crimproject.org/data/observations/').json()
# df_observations = pd.json_normalize(data_observations)
# df_observations.to_csv('../CRIM_Intervals_Notebooks/observations.csv')

In [30]:
df_observations = pd.read_csv('../CRIM_Intervals_Notebooks/observations.csv')
df_observations_test = df_observations[df_observations['piece.piece_id'] == 'CRIM_Model_0011'].copy()

In [31]:
viz.plot_comparison_heatmap(df_observations_test, 'ema', main_category='musical_type', other_category='observer.name', heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

## Generate networks with `create_comparisons_networks_and_interactive_df`

We can create a some network visualizations and a df to interact with patterns.

Work with both time interval and melodic interval!

**Important**:
- The plot will be saved in a html file in the folder you choose.
- The plot might not be displayed if it's not saved in the same folder as the Jupyter Notebook, but you should be able to find it and open it in your computer if necessary.

### time interval for pens

The columns of interest from table containing observations fetched from the CRIM database would be `mt_pe_tint` (mt= mechanical transformation, pe= periodic entries, tint=time intervals) and we would choose `interval_type` to be `time`. The table's corresponding ema column is `ema`.

In [22]:
pen_networks, pen_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_pe_tint', 'time', 'ema')

interactive(children=(Text(value='', description='search_pattern_starts_with'), Output()), _dom_classes=('widg…

**see all the avialable patterns**

In [23]:
pen_networks

{'all': <class 'pyvis.network.Network'> |N|=64 |E|=378,
 'nan': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 'B1': <class 'pyvis.network.Network'> |N|=4 |E|=8,
 'S1': <class 'pyvis.network.Network'> |N|=15 |E|=33,
 'S2': <class 'pyvis.network.Network'> |N|=2 |E|=2,
 'M1': <class 'pyvis.network.Network'> |N|=3 |E|=5,
 'S3': <class 'pyvis.network.Network'> |N|=2 |E|=5,
 'M8': <class 'pyvis.network.Network'> |N|=3 |E|=10,
 'M6': <class 'pyvis.network.Network'> |N|=2 |E|=4,
 'M2': <class 'pyvis.network.Network'> |N|=4 |E|=4,
 'M4': <class 'pyvis.network.Network'> |N|=6 |E|=14,
 'M3': <class 'pyvis.network.Network'> |N|=7 |E|=6,
 'S4': <class 'pyvis.network.Network'> |N|=3 |E|=27,
 'S6': <class 'pyvis.network.Network'> |N|=2 |E|=1,
 'B2': <class 'pyvis.network.Network'> |N|=2 |E|=22,
 'M5': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 'B3': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 'B4': <class 'pyvis.network.Network'> |N|=2 |E|=1,
 'L1': <class 'pyvis.network.Network'> |N|=2 |E|=13,


In [24]:
# show all the patterns mapped onto one network
pen_networks['all'].show('../CRIM_Intervals_Notebooks/all_time_pen.html')

In [25]:
# show only one family of patterns mapped onto one network
pen_networks['S1'].show('S1_time_pen.html')

### melodic intervals for fuga

The columns of interest from table containing observations fetched from the CRIM database would be `mt_fg_int` (mt= mechanical transformation, fg= Fuga, int=intervals) and we would choose `interval_type` to be `melodic`. The table's corresponding ema column is `ema`.

In [26]:
fuga_networks, fuga_widget = viz.create_comparisons_networks_and_interactive_df(df_observations, 'mt_fg_int', 'melodic', 'ema')

interactive(children=(Text(value='', description='search_pattern_starts_with'), Output()), _dom_classes=('widg…

In [27]:
# we are interested in the patterns that start with 4-
fuga_networks['4-'].show('../CRIM_Intervals_Notebooks/4-_fuga.html')

In [28]:
fuga_networks

{'all': <class 'pyvis.network.Network'> |N|=905 |E|=2,715,
 '4-': <class 'pyvis.network.Network'> |N|=68 |E|=137,
 '8-': <class 'pyvis.network.Network'> |N|=108 |E|=184,
 '5-': <class 'pyvis.network.Network'> |N|=123 |E|=293,
 'nan': <class 'pyvis.network.Network'> |N|=0 |E|=0,
 '4+': <class 'pyvis.network.Network'> |N|=117 |E|=206,
 '1+': <class 'pyvis.network.Network'> |N|=112 |E|=203,
 '8+': <class 'pyvis.network.Network'> |N|=92 |E|=259,
 '5+': <class 'pyvis.network.Network'> |N|=128 |E|=255,
 '12+': <class 'pyvis.network.Network'> |N|=19 |E|=25,
 '2+': <class 'pyvis.network.Network'> |N|=30 |E|=86,
 '12-': <class 'pyvis.network.Network'> |N|=25 |E|=27,
 '15-': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '6+': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '15+': <class 'pyvis.network.Network'> |N|=1 |E|=0,
 '2-': <class 'pyvis.network.Network'> |N|=3 |E|=4,
 '9+': <class 'pyvis.network.Network'> |N|=7 |E|=7,
 '7-': <class 'pyvis.network.Network'> |N|=3 |E|=2,
 '9-': <class 'pyvis

## `plot_relationship_network`

This method accepts a dataframe of CRIM relationships and generate a network of observations.

In the network:

- the nodes are the observations (labeled with the piece and measures they are in)

- the edges are the relationships. 
    - points from a model_observation to a derivative observation.
    - are weighted differently based on the type of the relationships.
    - are labeled with the relationship type
    - are colored based on the users' choice.

### WARNING: Edge color isn't always a correct indicator of the relationship type.

The color of the edges is derived from the nodes; therefore, they would not be colored correctly when:

1. color='derivative'

model1 --(quotation)--> model2

model3 --(new material)--> model2

because of the first relationship, model2 and the two edges pointing to it would be colored with quotation's color.

2. color='model'

model1 --(quotation)--> model2

model1 --(new material)--> model3

because of the first relationship, model1 and the two out edges from model1 would be colored with quotation's color

In this case, we should pay attention to the edges' thickness (different weights mean different relationship types) or hover over the edges to see the relationship type labels.

### Seeing the big picture with the color modes being `model` or `derivative`

We have two options for the `color` parameter. 

- `derivative` (default): the edges and nodes of derivative observations will be colored according to the relationships. 
- `model` (default): the edges and nodes of model observations will be colored according to the relationships. 

#### color=`derivative`

In [41]:
nt = viz.plot_relationship_network(df_relationships.head(300), color='derivative')
nt.show('../CRIM_Intervals_Notebooks/small_relationships_network_derivative.html')

#### color=`model`

In [42]:
nt2 = viz.plot_relationship_network(df_relationships.head(300), color='model')
nt2.show('small_relationships_network_model.html')

### Making uses of the other parameters.

The two plots are too cluttered so we can't see the relationship_type between specific measures inside pieces. 

#### Selecting specific models with `selected_model_ids` and specific derivatives with`selected_derivative_ids` 

In [43]:
nt3 = viz.plot_relationship_network(df_relationships, selected_derivative_ids=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt3.show('../CRIM_Intervals_Notebooks/selected_derivatives.html')

In [45]:
nt4 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0002', 'CRIM_Model_0016'], color='derivative')
nt4.show('selected_model.html')

#### Plotting pieces that are relevant to a list of interested pieces with `selected_members`
Having some of interest pieces, we want to plot all pieces that happen to connect to these pieces some how.

In [48]:
nt5 = viz.plot_relationship_network(df_relationships, selected_model_ids = ['CRIM_Model_0002', 'CRIM_Model_0016'], selected_families=['CRIM_Mass_0017_3', 'CRIM_Mass_0008_4',
       'CRIM_Mass_0003_5', 'CRIM_Mass_0014_4', 'CRIM_Mass_0018_1',
       'CRIM_Mass_0018_2', 'CRIM_Mass_0005_5'], color='derivative')
nt5.show('selected_model_families.html')

5 CRIM_Mass_0017_3, CRIM_Mass_0008_4, CRIM_Mass_0018_1, CRIM_Mass_0018_2, CRIM_Mass_0005_5 no longer exist in df because of other filtering options
